In [25]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import json
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.impute import SimpleImputer

# Caminho para o arquivo JSON
file_path = '/content/jogos_detalhe.json'

# ==============================================================================
# Seção 1: Carregamento e Preparação dos Dados
# ==============================================================================

# Carregar os dados do arquivo JSON
with open(file_path, 'r') as file:
    data = json.load(file)
df = pd.json_normalize(data)

# Pré-processamento: Conversão de listas em strings separadas por vírgula
cat_cols_list = ['mecanicas', 'categorias', 'temas']
for col in cat_cols_list:
    df[col] = df[col].apply(lambda x: ', '.join([str(i['nm_'+col[:-1]]) for i in x]) if isinstance(x, list) else x)

# Adicionando imputadores para colunas numéricas e categóricas
num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='constant', fill_value='desconhecido')

# Atualizando o preprocessor com imputadores
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[('imputer', num_imputer), ('scaler', StandardScaler())]), ['qt_jogadores_min', 'qt_jogadores_max', 'vl_tempo_jogo', 'idade_minima']),
        ('cat', Pipeline(steps=[('imputer', cat_imputer), ('encoder', OneHotEncoder(handle_unknown='ignore'))]), ['mecanicas', 'categorias', 'temas'])
    ])

# ==============================================================================
# Seção 2: Treinamento do Modelo de Clustering
# ==============================================================================

kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)  # Defina explicitamente n_init para evitar o aviso
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('cluster', kmeans)])

# Seleção das colunas relevantes para o clustering e treinamento do modelo
cols_for_clustering = ['qt_jogadores_min', 'qt_jogadores_max', 'vl_tempo_jogo', 'idade_minima', 'mecanicas', 'categorias', 'temas']
df_clustering = df[cols_for_clustering]
pipeline.fit(df_clustering)

# Adicionar a coluna de cluster ao DataFrame original
df['cluster'] = pipeline.predict(df_clustering)

# Definindo popularity_cols antes de calcular a pontuação de popularidade
popularity_cols = ['qt_quer', 'qt_favorito', 'qt_jogou', 'qt_tem', 'qt_teve']
df['popularity_score'] = df[popularity_cols].sum(axis=1)

# ==============================================================================
# Seção 3: Função de Recomendação
# ==============================================================================

def recomendar_jogos(base, tipo='nome', valor=''):
    """
    Função para recomendar jogos com base em diferentes critérios,
    excluindo o jogo usado como base para a pesquisa dos resultados.

    Parâmetros:
    - base (DataFrame): DataFrame contendo os dados dos jogos.
    - tipo (str): Tipo de recomendação ('nome', 'mecanica', 'categoria', 'tema').
    - valor (str): Valor correspondente ao tipo de recomendação.

    Retorna:
    - DataFrame com as top recomendações com base na popularidade, excluindo o jogo pesquisado.
    """
    if tipo != 'nome':
        valor = valor.lower()

    recomendacoes = pd.DataFrame()  # DataFrame vazio para casos sem recomendação

    if tipo == 'nome':
        filtro = base['nm_jogo'].str.lower() == valor.lower()
        if filtro.any():  # Verifica se alguma linha corresponde ao filtro
            jogo_selecionado = base[filtro].iloc[0]
            cluster = jogo_selecionado['cluster']
            # Excluir o jogo pesquisado das recomendações
            recomendacoes = base[(base['cluster'] == cluster) & (base['nm_jogo'].str.lower() != valor.lower())]
        else:
            print(f"Nenhum jogo encontrado com o nome: {valor}")
            return recomendacoes  # Retorna um DataFrame vazio
    elif tipo in ['mecanica', 'categoria', 'tema']:
        recomendacoes = base[base[tipo+'s'].str.lower().str.contains(valor)]
        if recomendacoes.empty:
            print(f"Nenhuma recomendação encontrada para {tipo}: {valor}")
            return recomendacoes

    if not recomendacoes.empty:
        top_recomendacoes = recomendacoes.sort_values(by='popularity_score', ascending=False).head(3)
        return top_recomendacoes[['nm_jogo', 'popularity_score']]
    else:
        return recomendacoes

# ==============================================================================
# Seção 4: Exemplos de Uso da Função de Recomendação
# ==============================================================================

# Exemplo de chamada da função
# resultado_nome = recomendar_jogos(df, tipo='nome', valor='Stone Age')
# print(resultado_nome)

# Exemplo de chamada da função para mecânica
# resultado_mecanica = recomendar_jogos(df, tipo='mecanica', valor='Alocação de Trabalhadores')
# print(resultado_mecanica)

# Nota: Remova os comentários das chamadas de função para executá-las e ver os resultados.

In [27]:
resultado_mecanica = recomendar_jogos(df, tipo='mecanica', valor='RPG')
print(resultado_mecanica)

                               nm_jogo  popularity_score
2002                   Eldritch Horror           12512.0
10204  Mansions of Madness (2ª Edição)           10335.0
8039                        Gloomhaven            6366.0


In [26]:
resultado_nome = recomendar_jogos(df, tipo='nome', valor='Catan: O Jogo')
print(resultado_nome)

            nm_jogo  popularity_score
598           Dixit           15235.0
766  The Resistance           14470.0
327        Munchkin           12097.0
